In [ ]:
# google image search example
# cando fm imagesize:600x600

In [2]:
from PIL import Image
import csv, json, os, zipfile

def ToResizedJPG(inpath, outJPGpath, outSize):
    img = Image.open(inpath)
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    newsize = (outSize, outSize)
    img = img.resize(newsize)
    img.save(outJPGpath)

csvFilePath = 'station_data.csv'
jsonFilePath = 'station_data.json'

header =  {"fields": [
        "id",
        "station",
        "name",
        "type",
        "logo",
        "genre",
        "broadcaster",
        "language",
        "country",
        "region",
        "bitrate",
        "format",
        "geo_fenced",
        "home_page",
        "reserved2"
    ], "stations": []}

# logoMissing, logoFile in CSV, deleted from json output - used for managing logos

newpath = "radio-logos"
isExist = os.path.exists(newpath)
if not isExist:
    os.makedirs(newpath)
    os.chdir(newpath)
    os.makedirs('thumbs')
    os.chdir('..')
    os.getcwd()

data = []
with open(csvFilePath) as csvFile:
    csvReader = csv.DictReader(csvFile)
    for rows in csvReader:
        logoFileSM = rows['logoFile'].replace('.jpg', '_sm.jpg').replace('.png', '_sm.png')
        if rows['logoMissing'] == 'TRUE':
            os.system(r'copy moode-default.jpg ".\radio-logos\{}.jpg"'.format(rows['name']))
            os.system(r'copy moode-default.jpg ".\radio-logos\thumbs\{}.jpg"'.format(rows['name']))
            os.system(r'copy moode-default.jpg ".\radio-logos\thumbs\{}_sm.jpg"'.format(rows['name']))
        elif ".png" in rows['logoFile']:
            ToResizedJPG(r".\600x600\{}.png".format(rows['name']), r".\radio-logos\{}.jpg".format(rows['name']), 600)
            ToResizedJPG(r".\600x600\{}.png".format(rows['name']), r".\radio-logos\thumbs\{}.jpg".format(rows['name']), 600)
            ToResizedJPG(r".\600x600\{}.png".format(rows['name']), r".\radio-logos\thumbs\{}_sm.jpg".format(rows['name']), 600)
        else:
            ToResizedJPG(r".\600x600\{}.jpg".format(rows['name']), r".\radio-logos\{}.jpg".format(rows['name']), 600)
            ToResizedJPG(r".\600x600\{}.jpg".format(rows['name']), r".\radio-logos\thumbs\{}.jpg".format(rows['name']), 600)
            ToResizedJPG(r".\600x600\{}.jpg".format(rows['name']), r".\radio-logos\thumbs\{}_sm.jpg".format(rows['name']), 600)    
        rows['id'] = int(rows['id']) 
        if '?' in rows['station']:
            rows['station'] = rows['station'] + '&station_name=' + rows['name'].replace('&', 'and').replace(' ', '_')
        else:
            rows['station'] = rows['station'] + '?station_name=' + rows['name'].replace('&', 'and').replace(' ', '_')
        del rows['logoMissing'] # exclude from json output
        del rows['logoFile']    # exclude from json output
        data.append(rows)

# update stations key values with data, writeout json
header["stations"] = data
headerjson = json.dumps(header, indent=4) #format json
with open(jsonFilePath, 'w') as jsonFile:
    jsonFile.write(headerjson)

# Create stations.zip matched required spec for moode audio
with zipfile.ZipFile('stations.zip', 'w') as zf:
    zf.write('station_data.json')
    for folderName, subfolders, filenames in os.walk('radio-logos'):
        for file in filenames:
            zf.write(os.path.join(folderName, file))


In [ ]:
# OLD CODE

        #else:  # 300, 200, 80
        #    os.system(r'copy ".\600x600\{0}" ".\radio-logos\{0}"'.format(rows['logoFile']))
        #    os.system(r'copy ".\600x600\{0}" ".\radio-logos\thumbs\{0}"'.format(rows['logoFile']))
        #    os.system(r'copy ".\600x600\{0}" ".\radio-logos\thumbs\{1}"'.format(rows['logoFile'], logoFileSM))